# Задание 1 - ЗАМОК

Выберите 2-3 значения лексемы по словарю (рекомендую использовать МАС, обязательно напишите, каким словарем пользовались). Для каждого значения придумайте несколько "диагностических" слов (которые сигнализируют о реализации одного из значений в контексте): например, ключ — вода / дверь. Подберите по НКРЯ контексты для выбранной лексемы, содержащие диагностические слова. Таким образом, получается размеченный тестсет.

Значения взяты из МАС

1.  Дворец и крепость феодала. Средневековый замок. □ На скале видны развалины какого-то замка. Пушкин, Путешествие в Арзрум. || Большой помещичий дом затейливой архитектуры. С одной стороны Обломовка, с другой — княжеский замок, с широким раздольем барской жизни. И. Гончаров, Обломов.

2. Устройство для запирания чего-л. ключом. Дверной замок. Висячий замок. Запереть на замок. || Приспособление для смыкания, соединения концов чего-л. (цепочки, браслета и пр.).

Слова диагносты: 
1. король, достопримечательность, каменный
2. сейф, дверь, ключ, щелкнуть

In [1]:
#Размеченные контексты из НКРЯ
data = {'Издали видны королевский замок Вавель и кафедральный собор Святых Вацлава и Станислава ― настоящие шедевры зодчества.':1,
'Часы Small Padlock из белого золота с бриллиантами внешне напоминают замок офисного сейфа.' : 2,
'В этот момент щелкнул замок и дверь чуть приоткрылась.' : 2, 
' Полез в ящик, нашёл старый врезной замок, пощелкал ключом туда-сюда.' : 2, 
'План был прост: взломать дверной замок, съесть в буфете все сосиски, взять из кассы всю мелочь и скрыться.' : 2,
' Замок щелкнул ― она дома.' : 2,
'В каждом старинном французском замке была комната короля, т. е. комната, которую занимал бы король, если бы он когда-нибудь посетил замок.' : 1, 
'Рядом с Монтре находится Шильонский замок, являющийся одной из самых посещаемых достопримечательностей Швейцарии.' : 1, 
'Итак, трёхэтажный, каменный тюремный замок в Старой Руссе был построен в 1885 году архитектором Василием Елкашевым на земле, принадлежащей солеваренному заводу, за счёт средств казны.' : 1
}

# Задание 2

Для каждого контекста определите значение автоматически — с помощью AdaGram / ELMo на выбор. В случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo — нужно кластеризовать.

In [2]:
import adagram
import pandas as pd
#from lxml import html
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import tqdm
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

In [3]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [4]:
vm.word_sense_probs('замок')

[(0, 0.21837189405487575),
 (1, 0.22392373681900282),
 (2, 0.23861078281656195),
 (3, 0.31909308952904347)]

In [5]:
vm.sense_neighbors('замок', 0)

[('орденский', 0, 0.6297175),
 ('монсегюр', 0, 0.615526),
 ('аббатство', 0, 0.6004314),
 ('дворец', 2, 0.5968682),
 ('сеньория', 0, 0.59168553),
 ('герцог', 2, 0.5900553),
 ('инстербург', 0, 0.57428056),
 ('каркассон', 0, 0.57125235),
 ('резиденция', 0, 0.5694655),
 ('амбуаз', 0, 0.56807244)]

In [6]:
vm.sense_neighbors('замок', 1)

[('задвижка', 0, 0.7730665),
 ('запор', 1, 0.74998415),
 ('засов', 1, 0.73436505),
 ('ключ', 0, 0.7206449),
 ('отмыкать', 0, 0.7102439),
 ('шпингалет', 0, 0.7052061),
 ('щеколда', 0, 0.70305103),
 ('защелка', 1, 0.70179313),
 ('запираться', 0, 0.696132),
 ('дверь', 6, 0.69068426)]

In [7]:
vm.sense_neighbors('замок', 2)

[('крепость', 1, 0.70243007),
 ('башня', 1, 0.6900965),
 ('бастион', 2, 0.66138613),
 ('средневековый', 1, 0.6602798),
 ('величественный', 1, 0.66022444),
 ('донжон', 0, 0.62887174),
 ('цитадель', 0, 0.62760615),
 ('башня', 3, 0.60489523),
 ('здание', 3, 0.6001993),
 ('крепостной', 0, 0.596294)]

In [8]:
for element in data.keys():
    means = vm.disambiguate('замок', normalize_pm(element))
    result = vm.sense_neighbors('замок', np.argmax(means))
    print(element)
    print(result)
    print()

Издали видны королевский замок Вавель и кафедральный собор Святых Вацлава и Станислава ― настоящие шедевры зодчества.
[('орденский', 0, 0.6297175), ('монсегюр', 0, 0.615526), ('аббатство', 0, 0.6004314), ('дворец', 2, 0.5968682), ('сеньория', 0, 0.59168553), ('герцог', 2, 0.5900553), ('инстербург', 0, 0.57428056), ('каркассон', 0, 0.57125235), ('резиденция', 0, 0.5694655), ('амбуаз', 0, 0.56807244)]

Часы Small Padlock из белого золота с бриллиантами внешне напоминают замок офисного сейфа.
[('крепость', 1, 0.70243007), ('башня', 1, 0.6900965), ('бастион', 2, 0.66138613), ('средневековый', 1, 0.6602798), ('величественный', 1, 0.66022444), ('донжон', 0, 0.62887174), ('цитадель', 0, 0.62760615), ('башня', 3, 0.60489523), ('здание', 3, 0.6001993), ('крепостной', 0, 0.596294)]

В этот момент щелкнул замок и дверь чуть приоткрылась.
[('задвижка', 0, 0.7730665), ('запор', 1, 0.74998415), ('засов', 1, 0.73436505), ('ключ', 0, 0.7206449), ('отмыкать', 0, 0.7102439), ('шпингалет', 0, 0.7052061),

Я бы сказала, что в адаграме одно значение словаря МАС разделено на два (там где ближайшие соседи "орденский" и "крепость"). Если считать их как одно значение, то все окей.

Еще можно обратить внимание на пример "Часы Small Padlock из белого золота с бриллиантами внешне напоминают замок офисного сейфа." - Тут идет сравнение, может, поэтому, адаграм неверно определил значение.

# Задание 3

Возьмите словарные толкования для каждого значения и припишите ему значение автоматически тем же способом, что и в п.2.

In [9]:
dictionary = { 'Дворец и крепость феодала.' : 1,
'Устройство для запирания чего-л. ключом.' : 2 }
for element in dictionary.keys():
    means = vm.disambiguate('замок', normalize_pm(element))
    result = vm.sense_neighbors('замок', np.argmax(means))
    print(element)
    print(result)
    print()

Дворец и крепость феодала.
[('орденский', 0, 0.6297175), ('монсегюр', 0, 0.615526), ('аббатство', 0, 0.6004314), ('дворец', 2, 0.5968682), ('сеньория', 0, 0.59168553), ('герцог', 2, 0.5900553), ('инстербург', 0, 0.57428056), ('каркассон', 0, 0.57125235), ('резиденция', 0, 0.5694655), ('амбуаз', 0, 0.56807244)]

Устройство для запирания чего-л. ключом.
[('задвижка', 0, 0.7730665), ('запор', 1, 0.74998415), ('засов', 1, 0.73436505), ('ключ', 0, 0.7206449), ('отмыкать', 0, 0.7102439), ('шпингалет', 0, 0.7052061), ('щеколда', 0, 0.70305103), ('защелка', 1, 0.70179313), ('запираться', 0, 0.696132), ('дверь', 6, 0.69068426)]



Определено верно

# Задание 4

Придумайте и реализуйте способ соотнести толкования с значениями с помощью тезауруса (Wiki_ru_wordnet / РуТез).

In [10]:
!pip install wiki-ru-wordnet

In [23]:
from wiki_ru_wordnet import WikiWordnet
import re
import networkx as nx
ruwn = WikiWordnet()

Я посмотрела на разные значения и выделила два, совпадающих с МАС

In [28]:
s = ruwn.get_synsets('замок')[0]
for w in s.get_words():
    print(w.lemma())
    print(w.definition())
    print()

замок
замок~ru~замок~ru~[[здание]] (или комплекс зданий), обычно обнесённое стеной и сочетающее в себе оборонительную и жилую функции (в наиболее распространённом значении — укреплённое [[жилище]] [[феодал]]а в средневековой Европе) {{пример|Почтенный {{выдел|за́мок}} был построен, // Как {{выдел|замки}} строиться должны: // Отменно прочен и спокоен // Во вкусе умной старины.|Пушкин|[[:s:Евгений Онегин. Глава 2 (Пушкин)|Евгений Онегин. Глава 2]]|1823|источник=source}}~44979~50915

цитадель
цитадель~ru~цитадель~ru~центральное, наиболее сильно фортифицированное укрепление в [[крепость|крепости]] или [[город]]е {{пример|Между {{выдел|цитаделью}} и городскими строениями находилось открытое незастроенное пространство, которое называлось эспланадой.}}~47018~977

крепость
крепость~ru~крепость~ru~{{военн.}} укреплённое [[место]] с долговременными оборонительными сооружениями, приспособленное для круговой обороны {{пример|{{выдел|Крепость}} оказалась, как {{выдел|крепость}},{{-}}четырехугольник

In [30]:
s2 = ruwn.get_synsets('замок')[2]
for w in s2.get_words():
    print(w.lemma())
    print(w.definition())
    print()

запор
запор~ru~запор~ru~[[устройство]], предназначенное для запирания чего-либо {{пример|Князь снял {{выдел|запор}}, отворил дверь и ― отступил в изумлении, весь даже вздрогнул: пред ним стояла Настасья Филипповна.|Ф. М. Достоевский|Идиот|1869|источник=НКРЯ}} {{пример|Потом он суёт руку под подушку и протягивает мне похожий на часы блестящий никелированный компас с крышкой, с {{выдел|запором}} и с вертящейся фосфорной картушкой.|А. П. Гайдар|Дым в лесу|1939|источник=НКРЯ}} {{пример|Главан ощупал {{выдел|запор}}, ― это был обычный железный засов на замке.|А. А. Фадеев|Молодая гвардия|1943–1951|источник=НКРЯ}}~75253~28230

замок
замок~ru~замок~ru~[[устройство]] для запирания чего-либо [[ключ]]ом {{пример|Дверной {{выдел|замок}}.}} {{пример|Запереть дверь на {{выдел|замок}}.}} {{пример|Ключ в {{выдел|замке}}.}} {{пример|Каморка запиралась на {{выдел|замо́к}}, напоминавший своим видом калач, только чёрный; ключ от этого {{выдел|замка́}} Герасим всегда носил с собой на пояске.|Тургенев|[[:s

# Задание 5

Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает с автоматически определенным значением толкования.